In [18]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collection import collections
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from webUtils import getHTML
from time import sleep
import time
from multiprocessing import Pool

################################################################################
## Music DB
################################################################################
from myMusicDBMap import myMusicDBMap
from mainDB import mainDB


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-06-14 21:10:47.779651


In [22]:
%load_ext autoreload
%autoreload
from mainDB import mainDB
mainDB = mainDB()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Creating DB Info For Discogs
Creating DB Info For AllMusic
Creating DB Info For MusicBrainz
Creating DB Info For AceBootlegs
Creating DB Info For RateYourMusic
Creating DB Info For LastFM
Creating DB Info For DatPiff
Creating DB Info For RockCorner
Creating DB Info For CDandLP
Creating DB Info For MusicStack
Creating DB Info For MetalStorm


In [ ]:
###### Parse Artists (Parallel)

In [15]:
def parseArtistsAM(modVal, force=False, doExtra=False):
    artsAM.parseArtistModValFiles(modVal, force=force)

def parseArtistsDC(modVal, force=False, doExtra=False):
    artsDC.parseArtistModValFiles(modVal, force=force)

def parseArtistsMB(modVal, force=False, doExtra=False):
    artsMB.parseArtistModValFiles(modVal, force=force)

def parseArtistsAB(modVal, force=False, doExtra=False):
    artsAB.parseArtistFiles(force=force)
    
def parseArtistsDP(modVal, force=False, doExtra=False):
    artsDP.parseArtistFiles(force=force)

def parseArtistsRM(modVal, force=False, doExtra=False):
    artsRM.parseArtistModValFiles(modVal, force=force)

def parseArtistsLM(modVal, force=False, doExtra=False):
    artsLM.parseArtistModValFiles(modVal, force=force)

def parseArtistsRC(modVal, force=False, doExtra=False):
    artsRC.parseArtistModValFiles(modVal, force=force)

def parseArtistsCL(modVal, force=False, doExtra=False):
    artsCL.parseArtistModValFiles(modVal, force=force)

def parseArtistsMS(modVal, force=False, doExtra=False):
    artsMS.parseArtistFiles(force=force)

def parseArtistsMT(modVal, force=False, doExtra=False):
    artsMT.parseArtistModValFiles(modVal, force=force)

    
def parseArtistsParallel(db, nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    if db == "Discogs":
        result = pool.map_async(parseArtistsDC, range(100))
    elif db == "AllMusic":
        result = pool.map_async(parseArtistsAM, range(100))
    elif db == "MusicBrainz":
        result = pool.map_async(parseArtistsMB, range(100))
    elif db == "AceBootlegs":
        result = pool.map_async(parseArtistsAB, [None])
    elif db == "DatPiff":
        result = pool.map_async(parseArtistsDP, [None])
    elif db == "RateYourMusic":
        result = pool.map_async(parseArtistsRM, range(100))
    elif db == "LastFM":
        result = pool.map_async(parseArtistsLM, range(100))
    elif db == "RockCorner":
        result = pool.map_async(parseArtistsRC, range(100))
    elif db == "CDandLP":
        result = pool.map_async(parseArtistsCL, range(100))
        #result = pool.map_async(parseArtistsCL, range(56,72))
        #result = pool.map_async(parseArtistsCL, [55,25,26])
    elif db == "MusicStack":
        result = pool.map_async(parseArtistsMS, [None])
    elif db == "MetalStorm":
        result = pool.map_async(parseArtistsMT, range(100))
    else:
        raise ValueError("[{0}] is not recognized as a DB".format(db))

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
parseArtistsParallel(db="AllMusic", nProcs=4)
parseArtistsParallel(db="MusicBrainz", nProcs=4)
parseArtistsParallel(db="Discogs", nProcs=3)
#parseArtistsParallel(db="AceBootlegs", nProcs=1)
parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=4)
parseArtistsParallel(db="LastFM", nProcs=3)
#parseArtistsParallel(db="RockCorner", nProcs=3)
#parseArtistsParallel(db="CDandLP", nProcs=3)
#parseArtistsParallel(db="MusicStack", nProcs=1)
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Sun Jun 14, 2020 20:17:13 for Parsing All Artists
Current Time is Sun Jun 14, 2020 20:17:13 for Parallel Artist Parsing
Parsing Artist Files For ModVal 21
Parsing Artist Files For ModVal 7
Parsing Artist Files For ModVal 0
Parsing Artist Files For ModVal 14
  ===> Found 362 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 15
  ===> Found 383 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 1
  ===> Found 386 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 22
  ===> Found 401 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 8
  ===> Found 361 files, but there are no new files to parse so skipping.
  ===> Found 361 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 23
Parsing Artist Files For ModVal 16
  ===> Found 365 files, but there are no new files to parse so skipping.
Parsing

  ===> Found 377 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 54
  ===> Found 388 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 58
removeFile("/Volumes/Piggy/Discog/artists-allmusic/33/0002569733.p")
removeFile("/Volumes/Piggy/Discog/artists-allmusic/39/0001278739.p")
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/33-DB.p
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/39-DB.p
  ===> Found 399 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 59
  ===> Found 415 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 55
  --> This file is 350.3kB.
Saving 367 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/39-Metadata.p
  --> This file is 17.1kB.
Saving 367 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/39-MediaMetadata.p
  ===> 

Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/90-DB.p
removeFile("/Volumes/Piggy/Discog/artists-allmusic/96/0000113396.p")
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/96-DB.p
  --> This file is 505.2kB.
Saving 388 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/90-Metadata.p
  --> This file is 17.9kB.
Saving 388 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/90-MediaMetadata.p
  --> This file is 321.9kB.
Saving 356 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/96-Metadata.p
  --> This file is 16.6kB.
Saving 356 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/96-MediaMetadata.p
  --> This file is 335.8kB.
  --> This file is 190.1kB.
Parsing Artist Files For ModVal 97
  ===> Found 385 files, but there are no new files to parse so skipping.

Parsing Artist Files For ModVal 14
Parsing Artist Files For ModVal 0
Parsi

  --> This file is 73.6kB.
Saving 959 new artist IDs media data to /Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/25-MediaMetadata.p
  --> This file is 687.1kB.
Saving 978 new artist IDs name data to /Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/19-Metadata.p
  --> This file is 75.0kB.
  --> This file is 280.9kB.
Parsing Artist Files For ModVal 26
Saving 978 new artist IDs media data to /Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/19-MediaMetadata.p
  --> This file is 728.7kB.
Saving 960 new artist IDs name data to /Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/5-Metadata.p
  --> This file is 73.5kB.
Saving 960 new artist IDs media data to /Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/5-MediaMetadata.p
  --> This file is 275.4kB.
Parsing Artist Files For ModVal 20
  --> This file is 308.7kB.
Parsing Artist Files For ModVal 6
  --> This file is 776.0kB.
Saving 963 new artist IDs name data to /Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/13-

  --> This file is 639.6kB.
Saving 927 new artist IDs name data to /Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/51-Metadata.p
  --> This file is 71.3kB.
Saving 927 new artist IDs media data to /Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/51-MediaMetadata.p
  --> This file is 308.2kB.
Parsing Artist Files For ModVal 41
  --> This file is 283.7kB.
Parsing Artist Files For ModVal 52
  ===> Found 931 files. There are 3 new files to parse.
  ===> Found 935 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 56
  ===> Found 947 files. There are 2 new files to parse.
  --> This file is 690.7kB.
Saving 946 new artist IDs name data to /Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/33-Metadata.p
  --> This file is 72.7kB.
Saving 946 new artist IDs media data to /Volumes/Piggy/Discog/artists-musicbrainz-db/metadata/33-MediaMetadata.p
  --> This file is 303.1kB.
Parsing Artist Files For ModVal 34
Saving 3 new artist IDs to /Volumes/Piggy

...

  ===> Found 7473 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 83
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-discogs-db/92-DB.p
  ===> Found 7470 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 84
  ===> Found 7460 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 85
  ===> Found 7365 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 86
  --> This file is 12.7MB.
Saving 7571 new artist IDs name data to /Volumes/Piggy/Discog/artists-discogs-db/metadata/92-Metadata.p
  --> This file is 436.7kB.
Saving 7571 new artist IDs media data to /Volumes/Piggy/Discog/artists-discogs-db/metadata/92-MediaMetadata.p
  --> This file is 4.1MB.
Parsing Artist Files For ModVal 93
  ===> Found 7588 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 87
  ===> Found 7374 files, but there are no new fil

Saving 576 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/17-Metadata.p
  --> This file is 39.1kB.
Saving 576 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/17-MediaMetadata.p
  --> This file is 152.8kB.
Saving 598 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/23-Metadata.p
  --> This file is 41.8kB.
Parsing Artist Files For ModVal 27
  --> This file is 39.1kB.
Saving 598 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/23-MediaMetadata.p
  --> This file is 162.4kB.
Saving 626 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/8-Metadata.p
  --> This file is 56.6kB.
Parsing Artist Files For ModVal 24
  --> This file is 40.8kB.
Saving 626 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/8-MediaMetadata.p
  --> This file is 62.1kB.
Parsing Artist Files For ModVal 36
  ===> Found 594 files, but there are no new file

  --> This file is 41.1kB.
Saving 628 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/34-MediaMetadata.p
  --> This file is 61.8kB.
Parsing Artist Files For ModVal 54
  --> This file is 66.7kB.
Parsing Artist Files For ModVal 35
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/49-DB.p
  ===> Found 601 files. There are 3 new files to parse.
  ===> Found 633 files. There are 1 new files to parse.
  --> This file is 170.9kB.
Saving 629 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/49-Metadata.p
  --> This file is 40.7kB.
Saving 629 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/49-MediaMetadata.p
  --> This file is 67.2kB.
Parsing Artist Files For ModVal 50
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/35-DB.p
Saving 3 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/54-DB.p
  ===> Found 631 files, but there are no new files to parse so skipping.
Pars

  --> This file is 37.9kB.
Saving 590 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/68-MediaMetadata.p
  --> This file is 53.5kB.
Parsing Artist Files For ModVal 69
  ===> Found 652 files. There are 1 new files to parse.
  ===> Found 588 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 61
  ===> Found 626 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 70
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/75-DB.p
  ===> Found 618 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 71
  ===> Found 628 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 62
  --> This file is 169.6kB.
Saving 652 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/75-Metadata.p
  --> This file is 43.0kB.
Saving 652 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/met

  ===> Found 610 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 98
  ===> Found 648 files. There are 1 new files to parse.
  ===> Found 625 files. There are 1 new files to parse.
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/87-DB.p
  --> This file is 178.9kB.
Saving 648 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/87-Metadata.p
Saving 1 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/98-DB.p
  --> This file is 42.5kB.
Saving 648 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/87-MediaMetadata.p
  --> This file is 71.6kB.
Parsing Artist Files For ModVal 88
  --> This file is 163.6kB.
Saving 625 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/98-Metadata.p
  --> This file is 40.4kB.
Saving 625 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/98-MediaMetadata.p
  --> This file is 62.9kB.
  ===> Foun

# Download My Known Artists

In [11]:
mdb = myMusicDBMap(debug=True)

Creating myMusicDBMap()
   Loading my music db map: /anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Counter({'Discogs': 3919, 'AllMusic': 3813, 'MusicBrainz': 3674, 'LastFM': 2777, 'RockCorner': 466, 'DatPiff': 458, 'AceBootlegs': 175, 'CDandLP': 80, 'RateYourMusic': 30, 'MusicStack': 5, 'MetalStorm': 0})


In [8]:
dbsToGet = ["AllMusic", "Discogs", "MusicBrainz", "LastFM", "RockCorner", "CDandLP"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist, dbdata["Discogs"]["Artists"].searchDiscogForArtist,
            dbdata["MusicBrainz"]["Artists"].searchMusicBrainzForArtist, dbdata["LastFM"]["Artists"].searchLastFMForArtist,
            dbdata["RockCorner"]["Artists"].searchRockCornerForArtist, dbdata["CDandLP"]["Artists"].searchCDandLPForArtist]

dbsToGet = ["AllMusic"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist]

In [9]:
def searchAll(mdb, minI=-1):
    nArtists = len(mdb.getArtists())
    for i,artistName in enumerate(mdb.getArtists()):
        if i <= minI:
            continue

        print('\n\n\n')
        print('-'*130)
        print(i,'/',nArtists,'===>\t',artistName)
        print('-'*130)
        print('\n')
        
        dbdata = mdb.getArtistData(artistName)
        if all([dbdata.get("LastFM"), dbdata.get("Discogs"), dbdata.get("AllMusic"), dbdata.get("MusicBrainz")]):
            print("{0} is well known.".format(artistName))
            continue
        else:
            print("{0} get it!".format(artistName))

        for search in searches:
            print("====>",search)
            try:
                search(artistName)
            except:
                sleep(3)

In [ ]:
searchAll(mdb, 3404)





----------------------------------------------------------------------------------------------------------------------------------
3405 / 4010 ===>	 Makoto Kawabata
----------------------------------------------------------------------------------------------------------------------------------


Makoto Kawabata get it!
====> <bound method artistsAM.searchAllMusicForArtist of <artistsAM.artistsAM object at 0x1a1a7df6a0>>


===================== Searching For Makoto Kawabata =====================
Found 18 artists
1 / 18 	: 0000667653 	 https://www.allmusic.com/artist/makoto-kawabata-mn0000667653/discography/all
2 / 18 	: 0001701162 	 https://www.allmusic.com/artist/makoto-kawabata-atsushi-tsuyama-yashimitsu-ichiraku-hiroshi-higashi-mn0001701162/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/62/0001701162.p (force=False)
  --> This file is 14.4kB.
Done. Sleeping for 2 seconds
3 / 18 	: 0000670021 	 https://www.allmusic.com/artist/makoto-ozone-mn0000670021/discography/a

Found 20 artists
1 / 20 	: 0001431056 	 https://www.allmusic.com/artist/martin-permer-mn0001431056/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/56/0001431056.p (force=False)
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds
2 / 20 	: 0002523048 	 https://www.allmusic.com/artist/juan-mart%C3%ADn-primera-mn0002523048/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/48/0002523048.p (force=False)
  --> This file is 15.4kB.
Done. Sleeping for 2 seconds
3 / 20 	: 0003289626 	 https://www.allmusic.com/artist/juan-mart%C3%ADn-primera-rocha-mn0003289626/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/26/0003289626.p (force=False)
  --> This file is 14.4kB.
Done. Sleeping for 2 seconds
4 / 20 	: 0000359026 	 https://www.allmusic.com/artist/ricky-martin-mn0000359026/discography/all
5 / 20 	: 0000813392 	 https://www.allmusic.com/artist/dean-martin-mn0000813392/discography/all
6 / 20 	: 0000043444 	 https://www.allmusic.com/artist/steve-martin-mn00

Found 19 artists
1 / 19 	: 0000601224 	 https://www.allmusic.com/artist/morphine-angel-mn0000601224/discography/all
2 / 19 	: 0002693655 	 https://www.allmusic.com/artist/angel-olsen-mn0002693655/discography/all
3 / 19 	: 0000601430 	 https://www.allmusic.com/artist/morphine-mn0000601430/discography/all
4 / 19 	: 0000230845 	 https://www.allmusic.com/artist/death-angel-mn0000230845/discography/all
5 / 19 	: 0000574654 	 https://www.allmusic.com/artist/dave-angel-mn0000574654/discography/all
6 / 19 	: 0000928351 	 https://www.allmusic.com/artist/morbid-angel-mn0000928351/discography/all
7 / 19 	: 0000585271 	 https://www.allmusic.com/artist/angel-romero-mn0000585271/discography/all
8 / 19 	: 0000754879 	 https://www.allmusic.com/artist/angel-witch-mn0000754879/discography/all
9 / 19 	: 0002966021 	 https://www.allmusic.com/artist/angel-haze-mn0002966021/discography/all
10 / 19 	: 0000044893 	 https://www.allmusic.com/artist/angel-viloria-mn0000044893/discography/all
11 / 19 	: 000177433

Saving /Volumes/Biggy/Discog/artists-allmusic/41/0003075241.p (force=False)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
20 / 20 	: 0003171114 	 https://www.allmusic.com/artist/nina-mn0003171114/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/14/0003171114.p (force=False)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds




----------------------------------------------------------------------------------------------------------------------------------
3423 / 4010 ===>	 Naturecore
----------------------------------------------------------------------------------------------------------------------------------


Naturecore get it!
====> <bound method artistsAM.searchAllMusicForArtist of <artistsAM.artistsAM object at 0x1a1a7df6a0>>


===================== Searching For Naturecore =====================
Found 5 artists
1 / 5 	: 0000980312 	 https://www.allmusic.com/artist/nightrockers-mn0000980312/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusi

19 / 20 	: 0002054947 	 https://www.allmusic.com/artist/carl-halens-1982-summa-cum-laude-orchestra-mn0002054947/discography/all
20 / 20 	: 0000536368 	 https://www.allmusic.com/artist/bud-freeman-his-summa-cum-laude-orchestra-mn0000536368/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/68/0000536368.p (force=False)
  --> This file is 14.6kB.
Done. Sleeping for 2 seconds




----------------------------------------------------------------------------------------------------------------------------------
3429 / 4010 ===>	 69 Tribe
----------------------------------------------------------------------------------------------------------------------------------


69 Tribe get it!
====> <bound method artistsAM.searchAllMusicForArtist of <artistsAM.artistsAM object at 0x1a1a7df6a0>>


===================== Searching For 69 Tribe =====================
Found 20 artists
1 / 20 	: 0000478982 	 https://www.allmusic.com/artist/a-tribe-called-quest-mn0000478982/discography/all
2 / 20 

...

Saving /Volumes/Biggy/Discog/artists-allmusic/58/0003902658.p (force=False)
  --> This file is 17.1kB.
Done. Sleeping for 2 seconds
17 / 20 	: 0000028458 	 https://www.allmusic.com/artist/terry-lee-brown-jr-mn0000028458/discography/all
18 / 20 	: 0001021274 	 https://www.allmusic.com/artist/wax-museums-mn0001021274/discography/all
19 / 20 	: 0001462223 	 https://www.allmusic.com/artist/wax-mannequin-mn0001462223/discography/all
20 / 20 	: 0003028304 	 https://www.allmusic.com/artist/wax-taylor-mn0003028304/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/4/0003028304.p (force=False)
  --> This file is 16.3kB.
Done. Sleeping for 2 seconds




----------------------------------------------------------------------------------------------------------------------------------
3589 / 4010 ===>	 Iggy Azalea
----------------------------------------------------------------------------------------------------------------------------------


Iggy Azalea get it!
====> <bound method art

Saving /Volumes/Biggy/Discog/artists-allmusic/46/0000459946.p (force=False)
  --> This file is 15.7kB.
Done. Sleeping for 2 seconds
20 / 20 	: 0001270728 	 https://www.allmusic.com/artist/yanco-inone-mn0001270728/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/28/0001270728.p (force=False)
  --> This file is 14.4kB.
Done. Sleeping for 2 seconds




----------------------------------------------------------------------------------------------------------------------------------
3591 / 4010 ===>	 Intro To India
----------------------------------------------------------------------------------------------------------------------------------


Intro To India get it!
====> <bound method artistsAM.searchAllMusicForArtist of <artistsAM.artistsAM object at 0x1a1a7df6a0>>


===================== Searching For Intro To India =====================
Found 2 artists
1 / 2 	: 0000772786 	 https://www.allmusic.com/artist/intro-to-india-mn0000772786/discography/all
Saving /Volumes/Biggy/D

Saving /Volumes/Biggy/Discog/artists-allmusic/66/0000455566.p (force=False)
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds
19 / 20 	: 0000471462 	 https://www.allmusic.com/artist/jerry-kinney-mn0000471462/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/62/0000471462.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
20 / 20 	: 0000490638 	 https://www.allmusic.com/artist/aaron-kinney-mn0000490638/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/38/0000490638.p (force=False)
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds




----------------------------------------------------------------------------------------------------------------------------------
3595 / 4010 ===>	 Ko & The Knockouts
----------------------------------------------------------------------------------------------------------------------------------


Ko & The Knockouts get it!
====> <bound method artistsAM.searchAllMusicForArtist of <artistsAM.artistsA

Found 20 artists
1 / 20 	: 0000465563 	 https://www.allmusic.com/artist/obliveon-mn0000465563/discography/all
2 / 20 	: 0000487111 	 https://www.allmusic.com/artist/oblivians-mn0000487111/discography/all
3 / 20 	: 0003810492 	 https://www.allmusic.com/artist/better-oblivion-community-center-mn0003810492/discography/all
4 / 20 	: 0001359617 	 https://www.allmusic.com/artist/jack-oblivian-mn0001359617/discography/all
5 / 20 	: 0003461111 	 https://www.allmusic.com/artist/heron-oblivion-mn0003461111/discography/all
6 / 20 	: 0000742828 	 https://www.allmusic.com/artist/source-direct-mn0000742828/discography/all
7 / 20 	: 0003028686 	 https://www.allmusic.com/artist/oblivion-mn0003028686/discography/all
8 / 20 	: 0000597582 	 https://www.allmusic.com/artist/agents-of-oblivion-mn0000597582/discography/all
9 / 20 	: 0000513734 	 https://www.allmusic.com/artist/oblivion-sun-mn0000513734/discography/all
10 / 20 	: 0002083181 	 https://www.allmusic.com/artist/oblivion-dust-mn0002083181/discogra

Found 1 artists
1 / 1 	: 0000284754 	 https://www.allmusic.com/artist/sam-cooke-the-soul-stirrers-mn0000284754/discography/all




----------------------------------------------------------------------------------------------------------------------------------
3609 / 4010 ===>	 Sam Smith
----------------------------------------------------------------------------------------------------------------------------------


Sam Smith get it!
====> <bound method artistsAM.searchAllMusicForArtist of <artistsAM.artistsAM object at 0x1a1a7df6a0>>


===================== Searching For Sam Smith =====================
Found 20 artists
1 / 20 	: 0002509942 	 https://www.allmusic.com/artist/sam-smith-mn0002509942/discography/all
2 / 20 	: 0001959850 	 https://www.allmusic.com/artist/sam-smith-mn0001959850/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/50/0001959850.p (force=False)
  --> This file is 15.4kB.
Done. Sleeping for 2 seconds
3 / 20 	: 0000917672 	 https://www.allmusic.com/a

In [ ]:
# Refused

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

In [ ]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

In [ ]:
from fsUtils import removeFile